# Array Computation with Dynamic Compute - GeoContexts
__________________
In previous notebooks we have explored the interactive visualization component of Dynamic Compute. Here we will dive into how and when we can extract the underlying pixel data associated with our [`Mosaic`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic) and [`ImageStack`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack) objects as numpy ndarrays.

For a more detailed look at all its classes and their available methods please visit the [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html) Documentation page.

In [ ]:
import descarteslabs as dl
import descarteslabs.dynamic_compute as dc
from descarteslabs.dynamic_compute import ImageStack, Mosaic

## GeoContexts 
Before we pull down any pixel data we must first define an area of interst (AOI) and output raster metadata parameters by which we organize our dataset. At Descartes Labs this is where the [`GeoContext`](https://docs.descarteslabs.com/descarteslabs/geo/readme.html) comes into play--with several operators to create and utilize geocontext objects, which we will introduce in this notebook.

First we'll define an interactive map as we have in other examples and an associated Sentinel-2 L2A mosaic and image stack. Here we will be returning to [Kossuth County, Iowa's Highest Corn Producing County](https://www.nass.usda.gov/Statistics_by_State/Iowa/Publications/County_Estimates/2021/IA-CtyEst-Corn-02-21.pdf).

In [ ]:
m = dc.map
m.center = 43.197541, -94.221831
m.zoom = 13

In [ ]:
s2_mosaic = dc.Mosaic.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "nir red green",
    start_datetime="2022-06-01",
    end_datetime="2022-7-01",
)

if int(dc.__version__[2]) >= 3:
    # this is the >=1.3.0 dynamic-compute version
    s2_stack = dc.ImageStack.from_product_bands(
        "esa:sentinel-2:l2a:v1",
        "nir red green",
        start_datetime="2022-06-01",
        end_datetime="2022-7-01",
    ).filter(dl.catalog.properties.cloud_fraction < 0.1)
else:
    # this is the <1.3.0 dynamic-compute version
    s2_stack = dc.ImageStack.from_product_bands(
        "esa:sentinel-2:l2a:v1",
        "nir red green",
        start_datetime="2022-06-01",
        end_datetime="2022-7-01",
    ).filter(lambda x: x.cloud_fraction < 0.1)


s2_mosaic.visualize("FCC", m)
s2_stack.median(axis="images").visualize("FCC-Cloudfree", m)

In [ ]:
m

## Interactive Map GeoContexts

The simplest geocontext to retrieve is that of your interactive map. We can simply call `map.geocontext()` to retrieve the current viewport as an [`AOI`](https://docs.descarteslabs.com/descarteslabs/geo/readme.html#descarteslabs.geo.AOI):

In [ ]:
geocontext = m.geocontext()
type(geocontext)

Note these raster metadata attributes, which may vary depending on the provenance of the particular object:
* __crs__: EPSG code
* __bounds__: bounding rectangle of the viewport
* __bounds_crs__: EPSG code
* __shape__: shape of the resulting array

In [ ]:
geocontext

## Putting the _Compute_ in Dynamic Compute
Once we have _either_ a mosaic or image stack _and_ a properly defined AOI we can now retrieve our pixel data as a numpy array.

To accomplish this we simply choose which bands we want to pull down and call either [`ImageStack.compute(aoi)`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.compute) or [`Mosaic.compute(aoi)`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic.compute).

Note that while resulting data type is a [`DotDict`](https://docs.descarteslabs.com/descarteslabs/utils/readme.html#descarteslabs.common.dotdict.dotdict.DotDict) we have _already retrieved the information we want_
* We can access the pixel data by calling __.ndarray__ on our results dictionary. The resulting array's shape will be __(nbands, ny, nx)__, as defined by the input geocontext
* We also have properties returned to us in our results dictionary, which we will return to later on

In [ ]:
s2_mosaic_data = s2_mosaic.pick_bands("nir red green").compute(geocontext)
type(s2_mosaic_data)
print(s2_mosaic_data.ndarray.shape)
print(s2_mosaic_data.properties)

And finally we can plot our dataset as an RGB:

In [ ]:
dl.utils.display(s2_mosaic_data.ndarray)

We note there is some pesky cloud and cloud shadows present in our scene, but also recall that we have the _temporal dimension_ exposed to us through our image stack. We can also pull down that _entire stack of data_ through computing our image stack. 

Note the resulting array's shape here will instead be __(nimages, nbands, ny, nx)__:

In [ ]:
s2_stack_data = s2_stack.pick_bands("nir red green").compute(geocontext)
s2_stack_data.ndarray.shape

Here we will return to our properties, where the metadata is much more useful than in our previous mosaic example. 

We can retrieve each image's date through inline list comprehension:

In [ ]:
props = s2_stack_data.properties
dates = [p["acquired"].strftime("%Y-%m-%d %HH-%MM-%SS") for p in props]
dates

In [ ]:
ids = [p["id"] for p in props]
ids

In [ ]:
titles = [f"{ids[i]} \n {dates[i]}" for i in range(len(dates))]

## Tying it All Together - ImageStacks
Note here that each image we retrieved in this stack _may not completely cover our input AOI_, that is because we have found an area _on the boundary between Sentinel-2 Imagery collections_. In the below plot we label each image with it's associated collectoin time as well as it's unique image ID:

In [ ]:
dl.utils.display(*s2_stack_data.ndarray[:3], title=titles[:3])

Note here that not each indvidual image covers the entire input AOI, this is because we are plotting individual Sentinel-2 scenes!

## Interoperability with other GeoContext Objects
In the next few cells we will demonstrate how you can compute over shapely geometries or [`DLTile`](https://docs.descarteslabs.com/descarteslabs/geo/readme.html#descarteslabs.geo.DLTile)s:

In [ ]:
from shapely.wkt import loads
import matplotlib.pyplot as plt

You can create and compute over an AOI generated from a shapely polygon:

In [ ]:
geom = loads(
    """POLYGON ((-95.54491138405865 41.34232959809853, 
    -95.52455234632363 41.34232959809853, 
    -95.52455234632363 41.35521625255075, 
    -95.54491138405865 41.35521625255075, 
    -95.54491138405865 41.34232959809853))"""
)
aoi = dl.geo.AOI(geom, resolution=10.0, crs="EPSG:3857")

Compute mean masked NDVI through our time period over our AOI:

In [ ]:
s2_mosaic_arr = s2_mosaic.compute(aoi).ndarray
dl.utils.display(s2_mosaic_arr, figsize=(5, 5))

Or also over DLTile objects:

In [ ]:
dltile = dl.geo.DLTile.from_latlon(
    41.34232959809853, -95.54491138405865, tilesize=512, pad=0, resolution=10.0
)

In [ ]:
s2_stack_arr = s2_stack.mean(axis="images").compute(dltile).ndarray
dl.utils.display(s2_stack_arr, figsize=(5, 5))